In [ ]:
import sys
import shutil
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import flopy

In [ ]:
mf6exe = 'mf6beta'

In [ ]:
def get_idomain(nlay, nrow, ncol, lx, lz, fx, fz):
    idomain = np.ones((nlay, nrow, ncol), dtype=np.int)
    x1 = fx * lx
    y1 = lz
    x2 = lx
    y2 = fz * lz
    slope = (y2 - y1) / (x2 - x1)
    b = y1 - slope * x1

    delr = lx / ncol
    delv = lz / nlay
    xcenters = np.linspace(delr/2, lx - delr/2, ncol)
    zcenters = np.linspace(lz - delv / 2, delv/2, nlay)

    for k in range(nlay):
        zc = zcenters[k]
        for j in range(ncol):
            xc = xcenters[j]
            zedge = slope * xc + b
            if zc > zedge:
                idomain[k, 0, j] = 0
                
    kidm0, iidmn0, jidmn0 = np.where(idomain == 0)
    for k, j in zip(kidm0, jidmn0):
        if idomain[k, 0, j] == 0 and idomain[k, 0, j-1] == 1:
            idomain[k, 0, j-1] = 2

    for k, j in zip(kidm0, jidmn0):
        if idomain[k, 0, j] == 0 and idomain[k+1, 0, j] == 1:
            idomain[k+1, 0, j] = 3
                
    return idomain

idomain = get_idomain(10, 1, 20, 2, 1, .5, .5)
plt.figure(figsize=(15, 10))
plt.imshow(idomain[:, 0, :])
plt.colorbar()

In [ ]:
def sinfunc(a, b, c, d, x):
    return a * np.sin(b * (x - c)) + d

def get_model(ws, name, nlay=10, ncol=20, 
              lx=2., lz=1., fx=0.5, fz=0.5, 
              sealevel=0.75, amplitude=0.15, frequency=4,
              wellfact=1.0,
              single_matrix=False, hhformulation=True,
              htimes=None, hidx=None, 
              ctimes=None, cidx=None, 
              verbosity_level=1):

    nrow = 1
    delr = lx / ncol
    delc = 1.
    top = lz
    delz = lz / nlay
    botm = list(top - np.arange(delz, nlay * delz + delz, delz))

    perlen = [0.25] + 1000 * [0.001]
    nper = len(perlen)
    nstp = [250] + 1000 * [1]
    tsmult = 1.
    steady = True
    tdis_rc = []
    for i in range(nper):
        tdis_rc.append((perlen[i], nstp[i], tsmult))
        
    nouter, ninner = 200, 50
    hclose, rclose, relax = 1e-7, 1e-5, 0.97

    # build MODFLOW 6 files
    sim = flopy.mf6.MFSimulation(sim_name=name, version='mf6',
                                 exe_name=mf6exe,
                                 verbosity_level=verbosity_level,
                                 sim_ws=ws)
    sim.name_file.continue_ = False
    
    # create tdis package
    tdis = flopy.mf6.ModflowTdis(sim, time_units='DAYS',
                                 nper=nper, perioddata=tdis_rc)

    # create gwf model
    gwfname = 'gwf_' + name
    gwtname = 'gwt_' + name
    
    # --------------------  FLOW --------------------

    gwf = flopy.mf6.ModflowGwf(sim, modelname=gwfname, newtonoptions=True)

    imsgwf = flopy.mf6.ModflowIms(sim, print_option='summary',
                                  csv_output_filerecord=gwfname + '.ims.csv',
                                  complexity='complex',
                                  outer_hclose=hclose,
                                  outer_maximum=nouter,
                                  inner_maximum=ninner,
                                  inner_hclose=hclose, 
                                  rcloserecord=rclose,
                                  linear_acceleration='BICGSTAB',
                                  scaling_method='NONE',
                                  reordering_method='NONE',
                                  relaxation_factor=relax,
                                  filename='{}.ims'.format(gwfname), 
                                  no_ptcrecord=True)
    if single_matrix:
        sim.register_ims_package(imsgwf, [gwfname, gwtname])
    else:
        sim.register_ims_package(imsgwf, [gwfname])

        
    idomain = get_idomain(nlay, nrow, ncol, lx, lz, fx=fx, fz=fz)
    dis = flopy.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol,
                                  delr=delr, delc=delc,
                                  top=top, botm=botm,
                                  idomain=idomain)

    # initial conditions
    ic = flopy.mf6.ModflowGwfic(gwf, strt=lz)

    # node property flow
    npf = flopy.mf6.ModflowGwfnpf(gwf, xt3doptions=False,
                                  save_flows=True,
                                  save_specific_discharge=True,
                                  icelltype=1,
                                  k=864.)
    
    sto = flopy.mf6.ModflowGwfsto(gwf, sy=0.35, iconvert=1,
                                  steady_state=[False], 
                                  transient=[True])

    hhoptions = False
    if hhformulation:
        hhoptions = [('HHFORMULATION'), ('LHS')]
    buy = flopy.mf6.ModflowGwfbuy(gwf, hhoptions=hhoptions)

    # drn and ghb
    kidx, iidx, jidx = np.where(idomain > 1)
    xcellcenters = gwf.modelgrid.xcellcenters
    zcellcenters = gwf.modelgrid.zcellcenters
    botm = dis.botm.get_data()
    dt = 0.001
    times = np.arange(dt, 1.0 + dt, dt)
    sealevelts = [sealevel] + list(sinfunc(amplitude, frequency * 2 * np.pi, 0, sealevel, times))
    ghbspd = {}
    drnspd = {}
    for kper in range(nper):
        if kper == 0:
            sl = sealevel
        else:
            sl = sealevelts[kper]
        ghblist = []
        drnlist = []
        for k, i, j in zip(kidx, iidx, jidx):
            zcell = zcellcenters[k, i, j]
            cond = 864. * (delz * delc) / (0.5 * delr)
            if zcell > sl:
                drnlist.append([(k, i, j), zcell, 864., 0.])
            else:
                ghblist.append([(k, i, j), sl, 864., 35., 1024.5])
        if len(ghblist) > 0:
            ghbspd[kper] = ghblist
        if len(drnlist) > 0:
            drnspd[kper] = drnlist
    
    # drn
    drn1 = flopy.mf6.ModflowGwfdrn(gwf,
                                   stress_period_data=drnspd,
                                   print_input=True,
                                   print_flows=True,
                                   save_flows=False,
                                   pname='DRN-1',
                                   auxiliary='CONCENTRATION')

    # ghb
    ghb1 = flopy.mf6.ModflowGwfghb(gwf,
                                   stress_period_data=ghbspd,
                                   print_input=True,
                                   print_flows=True,
                                   save_flows=False,
                                   pname='GHB-1',
                                   auxiliary=['CONCENTRATION', 'DENSITY'])
    
    wellist1 = []
    qwell = 5.7024 * wellfact
    qwell = qwell / nlay
    for k in range(nlay):
        wellist1.append([(k, 0, 0), qwell, 0.])
    wel1 = flopy.mf6.ModflowGwfwel(gwf,
                                   stress_period_data=wellist1,
                                   print_input=True,
                                   print_flows=True,
                                   save_flows=False,
                                   pname='WEL-1',
                                   auxiliary='CONCENTRATION',
                                   filename='{}.wel'.format(gwfname))

    # output control
    oc = flopy.mf6.ModflowGwfoc(gwf,
                                budget_filerecord='{}.cbc'.format(gwfname),
                                head_filerecord='{}.hds'.format(gwfname),
                                headprintrecord=[
                                    ('COLUMNS', 10, 'WIDTH', 15,
                                     'DIGITS', 6, 'GENERAL')],
                                saverecord=[('HEAD', 'ALL'),
                                            ('BUDGET', 'ALL')],
                                printrecord=[('HEAD', 'LAST'),
                                             ('BUDGET', 'ALL')])

    # --------------------  TRANSPORT --------------------
    
    # create gwt model
    gwt = flopy.mf6.MFModel(sim, model_type='gwt6', modelname=gwtname,
                            model_nam_file='{}.nam'.format(gwtname))

    if not single_matrix:
        imsgwt = flopy.mf6.ModflowIms(sim, print_option='all',
                                      outer_hclose=hclose,
                                      outer_maximum=10,
                                      inner_maximum=5,
                                      inner_hclose=hclose, 
                                      rcloserecord=rclose,
                                      linear_acceleration='BICGSTAB',
                                      scaling_method='NONE',
                                      reordering_method='NONE',
                                      relaxation_factor=relax,
                                      filename='{}.ims'.format(gwtname))
        sim.register_ims_package(imsgwt, [gwt.name])

    dis = flopy.mf6.ModflowGwtdis(gwt, nlay=nlay, nrow=nrow, ncol=ncol,
                                  delr=delr, delc=delc,
                                  top=top, botm=botm,
                                  idomain=idomain)

    # initial conditions
    ic = flopy.mf6.ModflowGwtic(gwt, strt=35.,
                                filename='{}.ic'.format(gwtname))

    # advection
    adv = flopy.mf6.ModflowGwtadv(gwt, scheme='UPSTREAM',
                                  filename='{}.adv'.format(gwtname))

    # dispersion
    diffc = 0.57024
    dsp = flopy.mf6.ModflowGwtdsp(gwt, xt3d=False, diffc=diffc,
                                  # alh=0., alv=0., ath=0., atv=0.,
                                  filename='{}.dsp'.format(gwtname))

    # mass storage and transfer
    porosity = 0.35
    mst = flopy.mf6.ModflowGwtmst(gwt, porosity=porosity,
                                  filename='{}.mst'.format(gwtname))

    # sources
    sourcerecarray = [('GHB-1', 'AUX', 'CONCENTRATION'),
                      ('WEL-1', 'AUX', 'CONCENTRATION')]
    ssm = flopy.mf6.ModflowGwtssm(gwt, sources=sourcerecarray,
                                  filename='{}.ssm'.format(gwtname))
    
    # fmi
    # this problem does not want to converge unless the flow correction
    # is implemented.  It looks like very small flow errors can wreak
    # havoc on the transport solution.  The correction seems to mitigate
    # these problems and at least keep the solution stable
    fmi = flopy.mf6.ModflowGwtfmi(gwt, flow_imbalance_correction=True)
    
    # output control
    oc = flopy.mf6.ModflowGwtoc(gwt,
                                budget_filerecord='{}.cbc'.format(gwtname),
                                concentration_filerecord='{}.ucn'.format(
                                    gwtname),
                                concentrationprintrecord=[
                                    ('COLUMNS', 10, 'WIDTH', 15,
                                     'DIGITS', 6, 'GENERAL')],
                                saverecord=[('CONCENTRATION', 'ALL')],
                                printrecord=[('CONCENTRATION', 'LAST'),
                                             ('BUDGET', 'ALL')])
    
    # --------------------  EXCHANGE --------------------

    # GWF GWT exchange
    gwfgwt = flopy.mf6.ModflowGwfgwt(sim, exgtype='GWF6-GWT6',
                                     exgmnamea=gwfname, exgmnameb=gwtname,
                                     filename='{}.gwfgwt'.format(name))

    # --------------------  WRITE AND RUN MODEL --------------------
    
    sim.write_simulation()
    v = sim.run_simulation(report=True, silent=False)
    #for idx in range(-3, 0):
    #    print(v[1][idx])

    # --------------------  LOAD RESULTS --------------------
    
    fname = os.path.join(ws, 'gwf_' + name + '.hds')
    headobj = flopy.utils.HeadFile(fname, precision='double')
    hlist = None
    hts = None
    if htimes is not None:
        times = headobj.get_times()
        hlist = []
        for t in htimes:
            if isinstance(t, str):
                if t.upper() == 'LAST':
                    tt = times[-1]
                    head = headobj.get_data(totim=tt)
                    hlist.append(head)
                if t.upper() == 'ALL':
                    for tt in times:
                        head = headobj.get_data(totim=tt)
                        hlist.append(head)
            else:
                head = headobj.get_data(totim=t)
                hlist.append(head)

    if hidx is not None:
        hts = headobj.get_ts(hidx)
    headobj.file.close()


    fname = os.path.join(ws, 'gwt_' + name + '.ucn')
    concobj = flopy.utils.HeadFile(fname, text='concentration', precision='double')
    clist = None
    cts = None
    if ctimes is not None:
        times = concobj.get_times()
        clist = []
        for t in ctimes:
            if isinstance(t, str):
                if t.upper() == 'LAST':
                    tt = times[-1]
                    conc = concobj.get_data(totim=tt)
                    clist.append(conc)
                if t.upper() == 'ALL':
                    for tt in times:
                        conc = concobj.get_data(totim=tt)
                        clist.append(conc)
            else:
                conc = concobj.get_data(totim=t)
                clist.append(conc)

    if cidx is not None:
        cts = concobj.get_ts(cidx)
    concobj.file.close()
    
    return sim, hlist, hts, clist, cts

In [ ]:
def get_patch_collection(modelgrid, head, conc, cmap='jet', zorder=None):
    # create patches for each cell
    xv, yv, zv = modelgrid.xyzvertices
    botm = modelgrid.botm
    patches = []
    for k in range(modelgrid.nlay):
        for j in range(modelgrid.ncol):
            x0 = xv[0, j]
            x1 = xv[0, j + 1]
            z0 = zv[k, 0, j]
            z0 = min(z0, head[k, 0, j])
            z0 = max(z0, botm[k, 0, j])
            z1 = zv[k + 1, 0, j]
            poly = [[x0, z0], [x1, z0], [x1, z1], [x0, z1], [x0, z0]]
            #print(poly)
            patch = matplotlib.patches.Polygon(poly, closed=True, edgecolor='k', facecolor='red')
            patches.append(patch)
    pc = matplotlib.collections.PatchCollection(patches, cmap=cmap, zorder=zorder)
    pc.set_array(conc.flatten())
    return pc

In [ ]:
# Hydraulic Head formulation
fx = 0.5
fz = 0.5
lx = 2
lz = 1
sealevel = 0.85
amplitude = 0.14
frequency = 4
wellfact = 0.25

dt = 0.001
times = np.arange(dt, 1.0 + dt, dt)
sealevelts = 250 * [sealevel] + list(sinfunc(amplitude, frequency * 2 * np.pi, 0, sealevel, times))
simtime = np.linspace(dt, 1.25 + dt, 1251)

ws = 'temp/pxx_henry_nr'
name = 'henry'
sim, hlist, hts, clist, cts = get_model(ws, name, nlay=40, ncol=80,
                                        fx=fx, fz=fz, 
                                        sealevel=sealevel, amplitude=amplitude, frequency=frequency,
                                        wellfact=wellfact,
                                        hhformulation=True,
                                        verbosity_level=0,
                                        htimes=['all'],
                                        ctimes=['all'])

In [ ]:
gwf = sim.get_model('gwf_' + name)
botm = gwf.modelgrid.botm
idomain = gwf.modelgrid.idomain
levels = [0.01, .1, .5, .9, .99]

#[249, 311, 436]
itime = len(clist) - 1
conc = clist[itime]
head = hlist[itime]
conc = np.ma.masked_greater(conc, 1e20)
conc = np.ma.masked_where(head < botm, conc)

fig = plt.figure(figsize=(15,9))
ax = fig.add_subplot(1, 1, 1, aspect='equal')

sl = sealevelts[itime]
seapoly = np.array([[lx*fx, sl], [lx, sl], [lx, 0]])
patch = matplotlib.patches.Polygon(seapoly, closed=True, facecolor='darkred', zorder=0)
ax.add_patch(patch)

aqpoly = np.array([[0, 0], [lx, 0], [lx, fz*lz], [lx*fx, lz], [0, lz]])
patch = matplotlib.patches.Polygon(aqpoly, closed=True, facecolor='.7', zorder=1)
ax.add_patch(patch)

xs = flopy.plot.PlotCrossSection(gwf, line={'row': 0})

pc = get_patch_collection(gwf.modelgrid, head, conc, zorder=2)
pc.set_clim(0, 35.)
ax.add_collection(pc)
xs.plot_grid(linewidths=0.5)

cs = plt.contour(np.flipud(conc[:, 0, :] / 35.), extent=[0, lx, 0, lz], 
                 levels=levels, colors='white', zorder=2)
ax.clabel(cs)
ax.set_xlabel('DISTANCE, IN METERS')
ax.set_ylabel('ELEVATION, IN METERS')
#ax.set_title('MODFLOW 6 HYDRAULIC HEAD FORMULATION')

# Seawater Level versus time plot

In [ ]:
fig = plt.figure(dpi=300)
ax = fig.add_subplot(1, 1, 1)
ax.plot(np.arange(dt, 1.25 + dt, dt), sealevelts)
ax.set_xlabel('TIME, IN DAYS', fontsize=6)
ax.set_ylabel('SEAWATER LEVEL, IN METERS', fontsize=6)

#fname = '../Figures/pxx_henrynr_sealevel.pdf'
#plt.savefig(fname, bbox_inches='tight')

## Results Plot

In [ ]:
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

def ax_formatter(ax):
    for tick in ax.xaxis.get_ticklabels():
        tick.set_fontsize(6)
        tick.set_fontname('Helvetica')
        tick.set_color('black')
        tick.set_weight('normal')    
    for tick in ax.yaxis.get_ticklabels():
        tick.set_fontsize(6)
        tick.set_fontname('Helvetica')
        tick.set_color('black')
        tick.set_weight('normal')
    ax.set_aspect(1.)
    ax.set_xticks([0, .5, 1., 1.5, 2.])
    ax.set_yticks([0, .25, .5, .75, 1])
    ax.xaxis.set_tick_params(width=0.5)
    ax.yaxis.set_tick_params(width=0.5)
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(0.5)

cmwidth = 8.25 # Groundwater 1 column width figure size
inwidth = cmwidth / 2.54
print('inwidth {}'.format(inwidth))
fig = plt.figure(figsize=(inwidth, 6.5), dpi=300)


# export figures in order to make an animation
gwf = sim.get_model('gwf_' + name)
botm = gwf.modelgrid.botm
idomain = gwf.modelgrid.idomain
levels = [0.01, .1, .5, .9, .99]

ws = './temp/pxx_henry_nr'
figdir = os.path.join(ws, 'figs')
if os.path.isdir(figdir):
    shutil.rmtree(figdir)
os.mkdir(figdir)

times2plot = [249, np.argmax(sealevelts), np.argmin(sealevelts)]
nplots = len(times2plot)

figtxt = ['(a)', '(b)', '(c)']

for ifig, itime in enumerate(times2plot):
    conc = clist[itime]
    head = hlist[itime]
    conc = np.ma.masked_greater(conc, 1e20)
    conc = np.ma.masked_where(head < botm, conc)

    ax = fig.add_subplot(nplots, 1, ifig + 1, aspect='equal')
    ax_formatter(ax)
    
    sl = sealevelts[itime]
    seapoly = np.array([[lx*fx, sl], [lx, sl], [lx, 0]])
    patch = matplotlib.patches.Polygon(seapoly, closed=True, facecolor='darkred', zorder=0)
    ax.add_patch(patch)

    aqpoly = np.array([[0, 0], [lx, 0], [lx, fz*lz], [lx*fx, lz], [0, lz]])
    patch = matplotlib.patches.Polygon(aqpoly, closed=True, facecolor='.7', zorder=1)
    ax.add_patch(patch)

    xs = flopy.plot.PlotCrossSection(gwf, line={'row': 0}, ax=ax)

    pc = get_patch_collection(gwf.modelgrid, head, conc, zorder=2)
    pc.set_clim(0, 35.)
    ax.add_collection(pc)
    
    xs.plot_grid(linewidths=0.5)

    cs = plt.contour(np.flipud(conc[:, 0, :] / 35.), extent=[0, lx, 0, lz], 
                     levels=levels, colors='white', zorder=2)
    ax.clabel(cs, fontsize=6, fmt='%1.2f')
    if ifig == nplots - 1:
        ax.set_xlabel('DISTANCE, IN METERS', fontsize=6)
    ax.set_ylabel('ELEVATION, IN METERS', fontsize=6)
    ttl = 'TIME = {:.3f} days'.format(simtime[itime])
    ax.set_title(ttl, fontsize=6)
    ax.text(1.9, 1.025, figtxt[ifig], fontsize=6)
    
#fname = '../Figures/pxx_henrynr_results.pdf'
#plt.savefig(fname, bbox_inches='tight')